## Fission and $\alpha$-emission decay rates

### Functions for basic nucleus properties

In [ ]:
import numpy as np

Define function to find radius of spherical nucleus.

In [ ]:
def average_radius(Z, A):
    return 1.07*A**(1./3) # F

__Mass of the constituents parts:__

$f_0(Z,A) = 1.007277Z + 1.008665(A-Z)$

In [ ]:
def f0(s, Z, A):
    return 1.00727647*Z + 1.00866492*(A - Z), 0.

__Volume:__

$f_1(Z,A) = -0.01691A$

In [ ]:
def f1(s, Z, A):
    return -0.01691*A, 0. # u

__Surface:__

$f_2(Z,A) = 0.01911 A^{2/3}$

In [ ]:
def f2(s, Z, A):
    return 0.01911*A**(2/3.), 2./5*0.01911*A**(2./3)*s**2 # u

__Coulomb:__

$f_3(Z,A) = 0.000763 Z^2 A^{-1/3}$

In [ ]:
def f3(s, Z, A):
    return 0.000763*Z**2/A**(1/3.), -1./5*0.000763*Z**2/A**(1./3)*s**2 # u

__Asymmetry:__

$f_4(Z,A) = 0.10175 (Z - A/2)^2 /A$

In [ ]:
def f4(s, Z, A):
    return 0.10175*(Z - A/2.)**2/A, 0. # u

__Pairing:__

$f_5(Z,A) = 0.012 A^{-1/2} \begin{pmatrix}-1\\0\\+1\end{pmatrix}$

In [ ]:
def f5(s, Z, A):
    if Z % 2 == 0 and (A - Z) % 2 == 0:
        return -0.012*A**(-1/2.), 0. # u
    elif Z % 2 == 1 and (A - Z) % 2 == 1:
        return 0.012*A**(-1/2.), 0. # u
    else:
        return 0., 0. # u

Define function to calculate mass of nucleus in Liquid Drop Model.

In [ ]:
def liquid_drop_mass(s, Z, A):
    return f0(s, Z, A)[0] + f1(s, Z, A)[0] + f2(s, Z, A)[0] + f2(s, Z, A)[1] \
            + f3(s, Z, A)[0] + f3(s, Z, A)[1] + f4(s, Z, A)[0] + f5(s, Z, A)[0]

### $\alpha$-emission decay rates

Define function to find kinetic energy of alpha-particle.

In [ ]:
u = 931.494103
mass_alpha = 4.001506179127

def alpha_energy(Z, A):
    
    # estimate masses of parent and daughter particles using liquid drop model
    mass_parent = liquid_drop_mass(0., Z, A)
    mass_daughter = liquid_drop_mass(0., Z - 2, A - 4)
    
    # estimate Q value of reaction
    ...
    
    return Q

Define function for shape of potential barrier.

In [ ]:
epsilon_0 = 55.26349406e-3 # e^2/MeV/F

def alpha_barrier(s, Z, A):
    
    # estimate radius at which alpha-particle leaves potenital of (new) daughter nucleus
    radius = average_radius(Z - 2, A - 4) + average_radius(2, 4)
    
    # estimate coulomb potential due to repulsion of alpha-particle from daughter nucleus
    ...
    
    # set potential to depth of nuclear potential within interaction radius
    potential[s < radius] = -50
    
    return potential

In [ ]:
s = np.arange(0, 100, 0.01)
p = alpha_barrier(s, 92, 235)
E = alpha_energy(92, 235)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import rc

# set up latex labels on plot (optional)
try:
    rc('text', usetex=True) # can try usetex=False
    rc('font', size=14)
    rc('legend', fontsize=14)
    rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
except:
    pass    

# create figure
fig, ax = plt.subplots(figsize=(7, 7))

# set axis labels and limits
ax.set_xlabel(r'$s$ (F)')
ax.set_ylabel(r'$V(s)$ (MeV)')

#ax.set_xlim([0,20])
#ax.set_ylim([-20,10])

ax.plot(s, p, 'k')
ax.plot(s, E*np.ones_like(s), 'b', linestyle='dashed')

plt.show()

Define function to determine tunneling probability.

In [ ]:
hbar = 6.58211956e-22 # MeV/s

def alpha_probability(s, p, A, E):
    
    ...

Define function to estimate the number of trials.

In [ ]:
def alpha_trials(Z, A, E):
    
    # estimate radius at which alpha-particle leaves potenital of (new) daughter nucleus
    radius = average_radius(Z - 2, A - 4) + average_radius(2, 4)
    
    # estimate the number of trials per unit time
    N = np.sqrt(2*E/(mass_alpha*u))/(2*radius * 1e-15)
    
    return N

Define function to calculate the decay rate.

In [ ]:
def alpha_decay_rate(s, p, Z, A, E):
    
    return alpha_trials(Z, A, E)*alpha_probability(s, p, A, E)

In [ ]:
rate = alpha_decay_rate(s, p, 92, 235, E)

Find the half-life:

### Fission decay rates

Define function to find change in binding energy as spherical nucleus deforms towards a peanut shape.

In [ ]:
u = 931.494103

def deformation_energy(s, Z, A):
    
    # estimate radius of undeformed spherical parent nucleus
    radius = average_radius(Z, A)
    
    # approximate separtion s as change in axial ratio
    alpha_2 = s/radius # a/b - 1
    
    # estimate mass of deformed parent nucleus using liquid drop model
    mass = liquid_drop_mass(alpha_2, Z, A)
    
    return mass * u

Define function to find change in binding energy between single and double nuclei.

In [ ]:
epsilon_0 = 55.26349406e-3 # e^2/MeV/F

def product_energy(s, Z, A):
  
    # estimate atomic and mass number of two daughter nuclei
    Z_1 = int(0.45*Z)
    Z_2 = Z - Z_1
    A_1 = int(0.45*A)
    A_2 = A - A_1
    
    # estimate mass of two spherical daughter nuclei using liquid drop model
    mass_1 = liquid_drop_mass(0., Z_1, A_1)
    mass_2 = liquid_drop_mass(0., Z_2, A_2) 
    
    # estimate coulomb potential due to repulsion of two daughter nuclei
    radius = average_radius(Z_1, A_1) + average_radius(Z_2, A_2)
    potential = Z_1*Z_2/(4*np.pi*epsilon_0*(s + radius)) # Mev

    return (mass_1 + mass_2) * u + potential

Define function to find the barrier potential function, combining the two limiting cases.

In [ ]:
power = 0.5

def fission_barrier(s, Z, A):
    
    # estimate the potential for the deforming parent nucleus
    parent = deformation_energy(s, Z, A)
    # estimate the potential for the two daughter nuclei
    daughter = product_energy(s, Z, A)
    
    # set equilibrium state potential of parent nucleus to zero energy
    parent = parent - deformation_energy(0., Z, A)
    daughter = daughter - deformation_energy(0., Z, A)
    
    # smooth two functions together (not physically based)
    potential = np.zeros_like(s)
    potential[daughter < 0] = daughter[daughter < 0]
    potential[daughter > 0] = 1./(1./parent[daughter > 0]**power + 1./daughter[daughter > 0]**power)**(1./power)
    
    return potential

In [ ]:
s = np.arange(0, 100, 0.01)
p = fission_barrier(s, 92, 235)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import rc

# set up latex labels on plot (optional)
try:
    rc('text', usetex=True) # can try usetex=False
    rc('font', size=14)
    rc('legend', fontsize=14)
    rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
except:
    pass    

# create figure
fig, ax = plt.subplots(figsize=(7, 7))

# set axis labels and limits
ax.set_xlabel(r'$s$')
ax.set_ylabel(r'$\Delta B$ (MeV)')

ax.set_xlim([0,20])
ax.set_ylim([-20,10])

ax.plot(s, p, 'k')
### ax.plot(s, q, 'r')

plt.show()

Define function for inverted quantum harmonic potential.

In [ ]:
def quantum_harmonic(s, p, A, omega):
    
    # find peak of fission barrier
    peak = np.max(p)
    s_peak = s[p == peak][0]
    
    return 0.5 * (A*1.66054e-27*omega**2) * (s*1e-15)**2 / 1.60218e-13

In [ ]:
q = quantum_harmonic(s, p, 235, 1e20)

Estimate the half-life to fission decay.

In [ ]:
omega = ...
halflife = 2*np.pi*np.log(2)*np.exp(2*np.pi*E*1.60218e-13/(1.054571817e-34*omega))

In [ ]:
halflife